In [ ]:
#hide
#skip
! [ -e /content ] && pip install -Uqq fastai  # upgrade fastai on colab

In [ ]:
#default_exp tabular.data

In [ ]:
#export
from fastai.torch_basics import *
from fastai.data.all import *
from fastai.tabular.core import *

In [ ]:
#hide
from nbdev.showdoc import *

# Tabular data

> Helper functions to get data in a `DataLoaders` in the tabular application and higher class `TabularDataLoaders`

The main class to get your data ready for model training is `TabularDataLoaders` and its factory methods. Checkout the [tabular tutorial](http://docs.fast.ai/tutorial.tabular) for examples of use.

## TabularDataLoaders -

In [ ]:
#export
class TabularDataLoaders(DataLoaders):
    "Basic wrapper around several `DataLoader`s with factory methods for tabular data"
    @classmethod
    @delegates(Tabular.dataloaders, but=["dl_type", "dl_kwargs"])
    def from_df(cls, 
        df:pd.DataFrame,
        path:(str,Path)='.', # Location of `df`, defaults to current working directory
        procs:list=None, # List of `TabularProc`s
        cat_names:list=None, # Column names pertaining to categorical variables
        cont_names:list=None, # Column names pertaining to continuous variables
        y_names:list=None, # Names of the dependent variables
        y_block:TransformBlock=None, # `TransformBlock` to use for the target(s)
        valid_idx:list=None, # List of indices to use for the validation set, defaults to a random split
        **kwargs
    ):
        "Create `TabularDataLoaders` from `df` in `path` using `procs`"
        if cat_names is None: cat_names = []
        if cont_names is None: cont_names = list(set(df)-set(L(cat_names))-set(L(y_names)))
        splits = RandomSplitter()(df) if valid_idx is None else IndexSplitter(valid_idx)(df)
        to = TabularPandas(df, procs, cat_names, cont_names, y_names, splits=splits, y_block=y_block)
        return to.dataloaders(path=path, **kwargs)

    @classmethod
    def from_csv(cls, 
        csv:(str,Path,io.BufferedReader), # A csv of training data
        skipinitialspace:bool=True, # Skip spaces after delimiter
        **kwargs
    ):
        "Create `TabularDataLoaders` from `csv` file in `path` using `procs`"
        return cls.from_df(pd.read_csv(csv, skipinitialspace=skipinitialspace), **kwargs)

    @delegates(TabDataLoader.__init__)
    def test_dl(self, 
        test_items, # Items to create new test `TabDataLoader` formatted the same as the training data
        rm_type_tfms=None, # Number of `Transform`s to be removed from `procs`
        process:bool=True, # Apply validation `TabularProc`s to `test_items` immediately
        inplace:bool=False, # Keep separate copy of original `test_items` in memory if `False`
        **kwargs
    ):
        "Create test `TabDataLoader` from `test_items` using validation `procs`"
        to = self.train_ds.new(test_items, inplace=inplace)
        if process: to.process()
        return self.valid.new(to, **kwargs)

Tabular._dbunch_type = TabularDataLoaders
TabularDataLoaders.from_csv = delegates(to=TabularDataLoaders.from_df)(TabularDataLoaders.from_csv)

This class should not be used directly, one of the factory methods should be preferred instead. All those factory methods accept as arguments:

- `cat_names`: the names of the categorical variables
- `cont_names`: the names of the continuous variables
- `y_names`: the names of the dependent variables
- `y_block`: the `TransformBlock` to use for the target
- `valid_idx`: the indices to use for the validation set (defaults to a random split otherwise)
- `bs`: the batch size
- `val_bs`: the batch size for the validation `DataLoader` (defaults to `bs`)
- `shuffle_train`: if we shuffle the training `DataLoader` or not
- `n`: overrides the numbers of elements in the dataset
- `device`: the PyTorch device to use (defaults to `default_device()`)

In [ ]:
show_doc(TabularDataLoaders.from_df)

<h4 id="TabularDataLoaders.from_df" class="doc_header"><code>TabularDataLoaders.from_df</code><a href="__main__.py#L4" class="source_link" style="float:right">[source]</a></h4>

> <code>TabularDataLoaders.from_df</code>(**`df`**:`DataFrame`, **`path`**:`Path'>)`=*`'.'`*, **`procs`**:`list`=*`None`*, **`cat_names`**:`list`=*`None`*, **`cont_names`**:`list`=*`None`*, **`y_names`**:`list`=*`None`*, **`y_block`**:[`TransformBlock`](/data.block.html#TransformBlock)=*`None`*, **`valid_idx`**:`list`=*`None`*, **`bs`**=*`64`*, **`shuffle_train`**=*`None`*, **`shuffle`**=*`True`*, **`val_shuffle`**=*`False`*, **`n`**=*`None`*, **`device`**=*`None`*, **`drop_last`**=*`None`*, **`val_bs`**=*`None`*)

Create [`TabularDataLoaders`](/tabular.data.html#TabularDataLoaders) from `df` in `path` using `procs`

Let's have a look on an example with the adult dataset:

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv', skipinitialspace=True)
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


In [ ]:
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [Categorify, FillMissing, Normalize]

In [ ]:
dls = TabularDataLoaders.from_df(df, path, procs=procs, cat_names=cat_names, cont_names=cont_names, 
                                 y_names="salary", valid_idx=list(range(800,1000)), bs=64)

In [ ]:
dls.show_batch()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary
0,Private,HS-grad,Married-civ-spouse,Adm-clerical,Husband,White,False,24.0,121312.998272,9.0,<50k
1,Private,HS-grad,Never-married,Other-service,Not-in-family,White,False,19.0,198320.000325,9.0,<50k
2,Private,Bachelors,Married-civ-spouse,Sales,Husband,White,False,66.0,169803.999308,13.0,>=50k
3,Private,HS-grad,Divorced,Adm-clerical,Unmarried,White,False,40.0,799280.980929,9.0,<50k
4,Local-gov,10th,Never-married,Other-service,Own-child,White,False,18.0,55658.003629,6.0,<50k
5,Private,HS-grad,Never-married,Handlers-cleaners,Other-relative,White,False,30.0,375827.003847,9.0,<50k
6,Private,Some-college,Never-married,Handlers-cleaners,Own-child,White,False,20.0,173723.999335,10.0,<50k
7,?,Some-college,Never-married,?,Own-child,White,False,21.0,107800.997986,10.0,<50k
8,Private,HS-grad,Never-married,Handlers-cleaners,Own-child,White,False,19.0,263338.000072,9.0,<50k
9,Private,Some-college,Married-civ-spouse,Tech-support,Husband,White,False,35.0,194590.999986,10.0,<50k


In [ ]:
show_doc(TabularDataLoaders.from_csv)

<h4 id="TabularDataLoaders.from_csv" class="doc_header"><code>TabularDataLoaders.from_csv</code><a href="__main__.py#L24" class="source_link" style="float:right">[source]</a></h4>

> <code>TabularDataLoaders.from_csv</code>(**`csv`**:`BufferedReader'>)`, **`skipinitialspace`**:`bool`=*`True`*, **`path`**:`Path'>)`=*`'.'`*, **`procs`**:`list`=*`None`*, **`cat_names`**:`list`=*`None`*, **`cont_names`**:`list`=*`None`*, **`y_names`**:`list`=*`None`*, **`y_block`**:[`TransformBlock`](/data.block.html#TransformBlock)=*`None`*, **`valid_idx`**:`list`=*`None`*, **`bs`**=*`64`*, **`shuffle_train`**=*`None`*, **`shuffle`**=*`True`*, **`val_shuffle`**=*`False`*, **`n`**=*`None`*, **`device`**=*`None`*, **`drop_last`**=*`None`*, **`val_bs`**=*`None`*)

Create [`TabularDataLoaders`](/tabular.data.html#TabularDataLoaders) from `csv` file in `path` using `procs`

In [ ]:
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [Categorify, FillMissing, Normalize]
dls = TabularDataLoaders.from_csv(path/'adult.csv', path=path, procs=procs, cat_names=cat_names, cont_names=cont_names, 
                                  y_names="salary", valid_idx=list(range(800,1000)), bs=64)

In [ ]:
show_doc(TabularDataLoaders.test_dl)

<h4 id="TabularDataLoaders.test_dl" class="doc_header"><code>TabularDataLoaders.test_dl</code><a href="__main__.py#L33" class="source_link" style="float:right">[source]</a></h4>

> <code>TabularDataLoaders.test_dl</code>(**`test_items`**, **`rm_type_tfms`**=*`None`*, **`process`**:`bool`=*`True`*, **`inplace`**:`bool`=*`False`*, **`bs`**=*`16`*, **`shuffle`**=*`False`*, **`after_batch`**=*`None`*, **`num_workers`**=*`0`*, **`verbose`**=*`False`*, **`do_setup`**=*`True`*, **`pin_memory`**=*`False`*, **`timeout`**=*`0`*, **`batch_size`**=*`None`*, **`drop_last`**=*`False`*, **`indexed`**=*`None`*, **`n`**=*`None`*, **`device`**=*`None`*, **`persistent_workers`**=*`False`*, **`wif`**=*`None`*, **`before_iter`**=*`None`*, **`after_item`**=*`None`*, **`before_batch`**=*`None`*, **`after_iter`**=*`None`*, **`create_batches`**=*`None`*, **`create_item`**=*`None`*, **`create_batch`**=*`None`*, **`retain`**=*`None`*, **`get_idxs`**=*`None`*, **`sample`**=*`None`*, **`shuffle_fn`**=*`None`*, **`do_batch`**=*`None`*)

Create test [`TabDataLoader`](/tabular.core.html#TabDataLoader) from `test_items` using validation `procs`

External structured data files can contain unexpected spaces, e.g. after a comma. We can see that in the first row of adult.csv `"49, Private,101320, ..."`. Often trimming is needed. Pandas has a convenient parameter `skipinitialspace` that is exposed by `TabularDataLoaders.from_csv()`. Otherwise category labels use for inference later such as `workclass`:`Private` will be categorized wrongly to *0* or `"#na#"` if training label was read as `" Private"`. Let's test this feature.

In [ ]:
test_data = {
    'age': [49], 
    'workclass': ['Private'], 
    'fnlwgt': [101320],
    'education': ['Assoc-acdm'], 
    'education-num': [12.0],
    'marital-status': ['Married-civ-spouse'], 
    'occupation': [''],
    'relationship': ['Wife'],
    'race': ['White'],
}
input = pd.DataFrame(test_data)
tdl = dls.test_dl(input)

test_ne(0, tdl.dataset.iloc[0]['workclass'])

## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_torch_core.ipynb.
Converted 01_layers.ipynb.
Converted 01a_losses.ipynb.
Converted 02_data.load.ipynb.
Converted 03_data.core.ipynb.
Converted 04_data.external.ipynb.
Converted 05_data.transforms.ipynb.
Converted 06_data.block.ipynb.
Converted 07_vision.core.ipynb.
Converted 08_vision.data.ipynb.
Converted 09_vision.augment.ipynb.
Converted 09b_vision.utils.ipynb.
Converted 09c_vision.widgets.ipynb.
Converted 10_tutorial.pets.ipynb.
Converted 10b_tutorial.albumentations.ipynb.
Converted 11_vision.models.xresnet.ipynb.
Converted 12_optimizer.ipynb.
Converted 13_callback.core.ipynb.
Converted 13a_learner.ipynb.
Converted 13b_metrics.ipynb.
Converted 14_callback.schedule.ipynb.
Converted 14a_callback.data.ipynb.
Converted 15_callback.hook.ipynb.
Converted 15a_vision.models.unet.ipynb.
Converted 16_callback.progress.ipynb.
Converted 17_callback.tracker.ipynb.
Converted 18_callback.fp16.ipynb.
Converted 18a_callback.training.ipynb.
Converted 18b_callback.preds.ipynb.
Converted 